In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os

import tensorflow as tf
from PIL import Image, ImageFile
from tensorflow.keras import layers, models  # type: ignore
from tensorflow.keras.preprocessing.image import ImageDataGenerator  # type: ignore
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau

ImageFile.LOAD_TRUNCATED_IMAGES = True
print("TensorFlow version:", tf.__version__)
print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))

TensorFlow version: 2.18.0
Num GPUs Available: 1


In [ ]:
def remove_corrupted_images(base_dir):
    count = 0
    for root, dirs, files in os.walk(base_dir):
        for file in files:
            if file.lower().endswith((".jpg", ".jpeg", ".png")):
                try:
                    filepath = os.path.join(root, file)
                    img = Image.open(filepath)
                    img.verify()  # This will raise an exception for corrupt files
                except Exception as e:
                    print(f"Removing corrupt image: {filepath} ({e})")
                    os.remove(filepath)
                    count += 1
    print(f"Removed {count} corrupted image(s)")

In [ ]:
# Run this once before training
remove_corrupted_images("Mushrooms")

Removed 0 corrupted image(s)


In [ ]:
IMG_HEIGHT, IMG_WIDTH = 224, 224
BATCH_SIZE = 32

# train_datagen = ImageDataGenerator(
#     rescale=1.0 / 255,
#     validation_split=0.2,
#     rotation_range=20,
#     zoom_range=0.2,
#     horizontal_flip=True,
# )

In [ ]:
# train_datagen = ImageDataGenerator(
#     rescale=1.0 / 255,
#     validation_split=0.2,
#     rotation_range=30,
#     width_shift_range=0.2,
#     height_shift_range=0.2,
#     shear_range=0.2,
#     zoom_range=0.3,
#     horizontal_flip=True,
#     fill_mode="nearest",
# )


train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    validation_split=0.2,
    rotation_range=40,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    zoom_range=0.4,
    horizontal_flip=True,
    brightness_range=[0.8, 1.2],
    fill_mode="nearest",
)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    "/content/drive/MyDrive/Colab Notebooks/Mushrooms",
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    subset="training",
)

val_generator = train_datagen.flow_from_directory(
    "/content/drive/MyDrive/Colab Notebooks/Mushrooms",
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    subset="validation",
)

Found 5375 images belonging to 9 classes.
Found 1339 images belonging to 9 classes.


In [ ]:
base_model = tf.keras.applications.MobileNetV2(
    input_shape=(IMG_HEIGHT, IMG_WIDTH, 3), include_top=False, weights="imagenet"
)

In [ ]:
unfreezed_layers = -100
for layer in base_model.layers[:unfreezed_layers]:  # freeze early 90% layers
    layer.trainable = False

for layer in base_model.layers[unfreezed_layers:]:  # unfreeze deeper high-level layers
    layer.trainable = True

# base_model = tf.keras.applications.EfficientNetB0(
#     input_shape=(IMG_HEIGHT, IMG_WIDTH, 3), include_top=False, weights="imagenet"
# )

In [ ]:
# Freeze base
base_model.trainable = True


# model = models.Sequential(
#     [
#         base_model,
#         layers.GlobalAveragePooling2D(),
#         layers.Dense(128, activation="relu"),
#         layers.Dropout(0.5),
#         layers.Dense(9, activation="softmax"),  # 9 classes
#     ]
# )

# model = models.Sequential(
#     [
#         base_model,
#         layers.GlobalAveragePooling2D(),
#         layers.Dense(256, activation="relu"),
#         layers.BatchNormalization(),
#         layers.Dropout(0.3),
#         layers.Dense(128, activation="relu"),
#         layers.Dropout(0.3),
#         layers.Dense(9, activation="softmax"),  # 9 classes
#     ]
# )

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(512, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(256, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(9, activation='softmax'),
])

In [ ]:
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

In [ ]:
history = model.fit(train_generator, validation_data=val_generator, epochs=30)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/30
168/168 ━━━━━━━━━━━━━━━━━━━━ 1065s 6s/step - accuracy: 0.2762 - loss: 2.6557 - val_accuracy: 0.2405 - val_loss: 13.2560
Epoch 2/30
168/168 ━━━━━━━━━━━━━━━━━━━━ 136s 809ms/step - accuracy: 0.4814 - loss: 1.6378 - val_accuracy: 0.2226 - val_loss: 4.5677
Epoch 3/30
168/168 ━━━━━━━━━━━━━━━━━━━━ 136s 810ms/step - accuracy: 0.5895 - loss: 1.2382 - val_accuracy: 0.3167 - val_loss: 6.2529
Epoch 4/30
168/168 ━━━━━━━━━━━━━━━━━━━━ 142s 811ms/step - accuracy: 0.6607 - loss: 1.0327 - val_accuracy: 0.2957 - val_loss: 8.3757
Epoch 5/30
168/168 ━━━━━━━━━━━━━━━━━━━━ 137s 816ms/step - accuracy: 0.7057 - loss: 0.8886 - val_accuracy: 0.3607 - val_loss: 4.6150
Epoch 6/30
168/168 ━━━━━━━━━━━━━━━━━━━━ 141s 808ms/step - accuracy: 0.7349 - loss: 0.7942 - val_accuracy: 0.4145 - val_loss: 4.4257
Epoch 7/30
168/168 ━━━━━━━━━━━━━━━━━━━━ 150s 893ms/step - accuracy: 0.7522 - loss: 0.7308 - val_accuracy: 0.3167 - val_loss: 5.3798
Epoch 8/30
168/168 ━━━━━━━━━━━━━━━━━━━━ 136s 807ms/step - accuracy: 0.7640 - 

In [ ]:
base_model.trainable = True

In [ ]:
optimizer = Adam(learning_rate=1e-4)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# model.compile(
#     optimizer=tf.keras.optimizers.Adam(1e-5),  # Lower LR
#     loss="categorical_crossentropy",
#     metrics=["accuracy"],
# )

In [ ]:
# lr_schedule = tf.keras.callbacks.ReduceLROnPlateau(
#     monitor="val_loss", factor=0.5, patience=3, verbose=1, min_lr=1e-7
# )

lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1)

In [ ]:
model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=20,
    callbacks=[lr_schedule],
)

Epoch 1/20
168/168 ━━━━━━━━━━━━━━━━━━━━ 191s 930ms/step - accuracy: 0.9056 - loss: 0.2964 - val_accuracy: 0.5721 - val_loss: 2.0890 - learning_rate: 1.0000e-04
Epoch 2/20
168/168 ━━━━━━━━━━━━━━━━━━━━ 139s 827ms/step - accuracy: 0.9274 - loss: 0.2308 - val_accuracy: 0.6154 - val_loss: 1.7279 - learning_rate: 1.0000e-04
Epoch 3/20
168/168 ━━━━━━━━━━━━━━━━━━━━ 153s 911ms/step - accuracy: 0.9395 - loss: 0.1996 - val_accuracy: 0.6804 - val_loss: 1.3281 - learning_rate: 1.0000e-04
Epoch 4/20
168/168 ━━━━━━━━━━━━━━━━━━━━ 138s 824ms/step - accuracy: 0.9415 - loss: 0.1917 - val_accuracy: 0.7110 - val_loss: 1.1837 - learning_rate: 1.0000e-04
Epoch 5/20
168/168 ━━━━━━━━━━━━━━━━━━━━ 138s 822ms/step - accuracy: 0.9408 - loss: 0.1731 - val_accuracy: 0.7282 - val_loss: 1.0972 - learning_rate: 1.0000e-04
Epoch 6/20
168/168 ━━━━━━━━━━━━━━━━━━━━ 138s 821ms/step - accuracy: 0.9453 - loss: 0.1723 - val_accuracy: 0.7267 - val_loss: 1.0999 - learning_rate: 1.0000e-04
Epoch 7/20
168/168 ━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
def get_incremental_filename(base_name, extension):
    counter = 1
    while True:
        filename = f"{base_name}_{counter}.{extension}"
        if not os.path.exists(filename):
            return filename
        counter += 1

In [ ]:
# Usage
filename = get_incremental_filename("my_model", "h5")
model.save(filename)

model.save(f"/content/drive/MyDrive/Colab Notebooks/{filename}")
print(f"Model saved as {filename}")

Model saved as my_model_2.h5


In [ ]:
import pandas as pd
import tensorflow as tf
import os
from PIL import ImageFile

ImageFile.LOAD_TRUNCATED_IMAGES = True  # Prevent crash on corrupted images

def load_model_weights(model_path, weights=None):
    model = tf.keras.models.load_model(model_path)
    model.summary()
    return model

def decode_img(img_path, img_height, img_width):
    img = tf.io.read_file(img_path)
    img = tf.io.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, [img_height, img_width])
    img = tf.cast(img, tf.float32) / 255.0
    return img

def get_images_labels(df, classes, img_height, img_width):
    class_list = sorted(list(classes))
    label_map = {label: idx for idx, label in enumerate(class_list)}

    image_tensors = []
    label_tensors = []

    for _, row in df.iterrows():
        img_path = row["image_path"]
        label = row["label"]

        if not os.path.exists(img_path):
            print(f"Warning: File not found: {img_path}")
            continue

        try:
            img = decode_img(img_path, img_height, img_width)
            one_hot = tf.keras.utils.to_categorical(
                label_map[label], num_classes=len(classes)
            )
            image_tensors.append(img)
            label_tensors.append(one_hot)
        except Exception as e:
            print(f"Skipping {img_path}: {e}")
            continue

    if not image_tensors or not label_tensors:
        print("ERROR: No valid test images loaded!")
        return None, None

    test_images = tf.stack(image_tensors)
    test_labels = tf.convert_to_tensor(label_tensors)
    return test_images, test_labels

# Define paths directly since argparse doesn't work well in Colab
model_path = "/content/my_model_2.h5"
test_csv_path = "/content/drive/MyDrive/Colab Notebooks/sample_test_data/mushrooms_test.csv"

# Load test data
test_df = pd.read_csv(test_csv_path)
classes = {
    "Agaricus",
    "Amanita",
    "Boletus",
    "Cortinarius",
    "Entoloma",
    "Hygrocybe",
    "Lactarius",
    "Russula",
    "Suillus",
}

IMG_HEIGHT, IMG_WIDTH = 224, 224
print("Loading test images and labels...")
test_images, test_labels = get_images_labels(
    test_df, classes, IMG_HEIGHT, IMG_WIDTH
)

if test_images is None or test_labels is None:
    print("Exiting due to missing or invalid test data.")
else:
    print("Loading model...")
    my_model = load_model_weights(model_path)

    print("Evaluating model...")
    loss, acc = my_model.evaluate(test_images, test_labels, verbose=2)
    print("Test model accuracy: {:5.2f}%".format(100 * acc))

Loading test images and labels...
Loading model...


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_3      │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 512)            │       655,872 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 512)            │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 9)              │         2,313 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,050,571 (11.64 MB)

 Trainable params: 2,959,433 (11.29 MB)

 Non-trainable params: 91,136 (356.00 KB)

 Optimizer params: 2 (12.00 B)

Evaluating model...


1/1 - 4s - 4s/step - accuracy: 1.0000 - loss: 0.1654
Test model accuracy: 100.00%


In [ ]:
import matplotlib.pyplot as plt

def plot_training_history(history):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs = range(len(acc))

    plt.figure(figsize=(14, 5))

    # Accuracy
    plt.subplot(1, 2, 1)
    plt.plot(epochs, acc, 'b', label='Training acc')
    plt.plot(epochs, val_acc, 'r', label='Validation acc')
    plt.title('Training and Validation Accuracy')
    plt.legend()

    # Loss
    plt.subplot(1, 2, 2)
    plt.plot(epochs, loss, 'b', label='Training loss')
    plt.plot(epochs, val_loss, 'r', label='Validation loss')
    plt.title('Training and Validation Loss')
    plt.legend()

    plt.show()

# Example usage after training:
plot_training_history(history)

NameError: name 'history' is not defined